In [1]:
using Revise
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing
include("SISEPUEDEPJSF.jl")
import .SISEPUEDEPJSF



using Pkg
Pkg.activate(".")


using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP
using Gurobi
using GAMS

include("setup_analysis.jl")

  Activating project at `~/Documents/Projects/git_jbus/lac_decarbonization/julia`
┌ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72]
└ @ Base loading.jl:1664
  ** incremental compilation may be fatally broken for this module **

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1664
┌ Info: Precompiling GAMS [1ca51c6a-1b4d-4546-9ae1-53e0a243ab12]
└ @ Base loading.jl:1664
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1664


"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"

In [2]:
?NemoMod.calculatescenario

```
calculatescenario(dbpath::String; jumpmodel::JuMP.Model = Model(Cbc.Optimizer),
    calcyears::Array{Int, 1} = Array{Int, 1}(),
    varstosave::String = "vdemandnn, vnewcapacity, vtotalcapacityannual,
        vproductionbytechnologyannual, vproductionnn, vusebytechnologyannual,
        vusenn, vtotaldiscountedcost",
    restrictvars::Bool = true, reportzeros::Bool = false,
    continuoustransmission::Bool = false,
    forcemip::Bool = false, startvalsdbpath::String = "",
    startvalsvars::String = "", precalcresultspath::String = "",
    quiet::Bool = false
)
```

Calculates a scenario specified in a scenario database. Returns a `MathOptInterface.TerminationStatusCode` indicating the termination status reported by the solver used for the calculation (e.g., `OPTIMAL::TerminationStatusCode = 1`).

# Arguments

  * `dbpath::String`: Path to the scenario database, which must be a SQLite version 3 database that   implements NEMO's scenario database structure. See NEMO's documentation on scenario databases   for details. Empty scenario databases can be generated with NEMO's `createnemodb` function.
  * `jumpmodel::JuMP.Model`: [JuMP](https://github.com/jump-dev/JuMP.jl) model object   specifying the solver to be used for the calculation.   Examples: `Model(optimizer_with_attributes(GLPK.Optimizer, "presolve" => true))`,   `Model(CPLEX.Optimizer)`, `Model(optimizer_with_attributes(Gurobi.Optimizer, "NumericFocus" => 1))`.   Note that the solver's Julia package (Julia wrapper) must be installed. See the   documentation for JuMP for information on how to specify a solver and set solver options.
  * `calcyears::Array{Int, 1}`: Years to include in the calculation (a subset of the years specified in   the scenario database). All years in the database are included if this argument is omitted.
  * `varstosave::String`: Comma-delimited list of output variables whose results should be   saved in the scenario database when the scenario is calculated. See NEMO's documentation on   outputs for information on the variables that are available.
  * `restrictvars::Bool`: Indicates whether NEMO should conduct additional data analysis   to limit the set of variables created in the optimization problem for the scenario.   By default, to improve performance, NEMO selectively creates certain variables to   avoid combinations of subscripts that do not exist in the scenario's data. This option   increases the stringency of this filtering. It requires more processing time as the   problem is built, but it can substantially reduce the solve time for large models.
  * `reportzeros::Bool`: Indicates whether results saved in the scenario database should   include values equal to zero. Specifying `false` can substantially improve the   performance of large models.
  * `continuoustransmission::Bool`: Indicates whether continuous (`true`) or binary (`false`)   variables are used to represent investment decisions for candidate transmission lines. Not   relevant in scenarios that do not model transmission.
  * `forcemip::Bool`: Forces NEMO to formulate the optimization problem for the scenario as a   mixed-integer problem. This can improve performance with some solvers (e.g., CPLEX, Mosek). If   this option is set to `false`, the input parameters for the scenario (i.e., in the scenario   database) determine whether the optimization problem is mixed-integer.
  * `startvalsdbpath::String`: Path to a previously calculated scenario database from which NEMO   should take starting values for variables in the optimization problem formulated in this   function. This argument is used in conjunction with `startvalsvars`.
  * `startvalsvars::String`: Comma-delimited list of variables for which starting values should be set.   See NEMO's documentation on outputs for information on the variables that are available. NEMO   takes starting values from output variable results saved in the database identified by   `startvalsdbpath`. Saved results are matched to variables in the optimization problem using   the variables' subscripts, and starting values are set with JuMP's `set_start_value` function.   If `startvalsvars` is an empty string, NEMO sets starting values for all variables present in   both the optimization problem and the `startvalsdbpath` database.
  * `precalcresultspath::String`: Path to a previously calculated scenario database that NEMO should   copy over the database specified by `dbpath`. This argument can also be a directory containing   previously calculated scenario databases, in which case NEMO copies any file in the directory   with the same name as the `dbpath` database. The intent of the argument is to short-circuit   calculations in situations where valid results already exist.
  * `quiet::Bool`: Suppresses low-priority status messages (which are otherwise printed to   `STDOUT`).


In [122]:
fp_sqlite = "/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/test_nemo/nemomod_testdb.sqlite"
db = SQLite.DB(fp_sqlite)


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/test_nemo/nemomod_testdb.sqlite")

In [123]:
model = Model(Gurobi.Optimizer); set_silent(model)
@time NemoMod.calculatescenario(
    fp_sqlite; 
    jumpmodel = model,
    numprocs = 1
)





2023-17-Jan 23:43:05.545 Started modeling scenario.
2023-17-Jan 23:43:05.545 Validated run-time arguments.
2023-17-Jan 23:43:05.546 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/test_nemo/nemomod_testdb.sqlite.
2023-17-Jan 23:43:05.553 Dropped pre-existing result tables from database.
2023-17-Jan 23:43:05.553 Verified that transmission modeling is not enabled.
2023-17-Jan 23:43:05.591 Created parameter views and indices.
2023-17-Jan 23:43:05.595 Created temporary tables.
2023-17-Jan 23:43:05.744 Executed core database queries.
2023-17-Jan 23:43:05.747 Defined dimensions.
2023-17-Jan 23:43:05.748 Defined demand variables.
2023-17-Jan 23:43:05.750 Defined storage variables.
2023-17-Jan 23:43:05.751 Defined capacity variables.
2023-17-Jan 23:43:06.060 Defined activity variables.
2023-17-Jan 23:43:06.060 Defined costing variables.
2023-17-Jan 23:43:06.061 Defined reserve margin variables.
2023-17-Jan 23:43:06.061 Defined emissions v

OPTIMAL::TerminationStatusCode = 1

In [124]:
dict_tables = Dict()

tabs = SQLite.tables(db)[1]
table_name = 0
tabs_populated = Vector{String}()
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    
    (nrow(df_tmp) > 0) ? push!(tabs_populated, table_name) : nothing
end


In [125]:
get(dict_tables, "DefaultParams", nothing);

In [126]:
filter(x -> (x[:f] != "fuel_electricity"), get(dict_tables, "SpecifiedAnnualDemand", nothing))

,id,r,f,y,val
,Int64,String,String,String,Float64
1,4,costa_rica,fuel_oil,1000,72.0
2,5,costa_rica,fuel_oil,1001,73.5
3,6,costa_rica,fuel_oil,1002,75.0


In [99]:
sort([x for x in keys(dict_tables) if String(x[1:1]) == "v"])


8-element Vector{String}:
 "vdemandnn"
 "vnewcapacity"
 "vproductionbytechnologyannual"
 "vproductionnn"
 "vtotalcapacityannual"
 "vtotaldiscountedcost"
 "vusebytechnologyannual"
 "vusenn"

In [127]:
get(dict_tables, "MinShareProduction", nothing)

,id,r,t,f,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1000,0.0
2,2,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1001,0.15
3,3,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,0.3


In [130]:
t1 = get(dict_tables, "vproductionbytechnologyannual", nothing)
#filter(x -> (x[:t] == "pp_oil"), t1)
filter(x -> (x[:y] == "1002"), t1)


,r,t,f,y,val
,String,String,String,String,Float64
1,costa_rica,supply_fuel_crude,fuel_crude,1002,3.15576e11
2,costa_rica,pp_hydropower,fuel_electricity,1002,13.7627
3,costa_rica,pp_solar,fuel_electricity,1002,1.21603
4,costa_rica,st_compressed_air,fuel_electricity,1002,14.9759
5,costa_rica,st_batteries,fuel_electricity,1002,14.9787
6,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,22.5
7,costa_rica,st_flywheels,fuel_electricity,1002,14.9787
8,costa_rica,supply_fuel_solar,fuel_solar,1002,4.8255
9,costa_rica,supply_fuel_oil,fuel_oil,1002,52.5


In [139]:
(24*365*0.75*0.085)*1.298

724.8681

In [109]:
tab = filter(x -> ((x[:y] == "1002") & (x[:f] == "fuel_electricity")), t1)
sum(tab[:, :val]) - 21.7722

37.37555310931303

In [67]:
t2 = get(dict_tables, "InputActivityRatio", nothing)
filter(x -> ((x[:y] == "1000") & (x[:t] == "pp_oil")), t2)
166.262 - 36.947*2.5
37.2041*2.5

93.01024999999998

In [105]:
t3 = get(dict_tables, "SpecifiedAnnualDemand", nothing)
#filter(x -> ((x[:y] == "1001") & (x[:f] == "fuel_oil")), t3)
filter(x -> ((x[:y] == "1002")), t3)


,id,r,f,y,val
,Int64,String,String,String,Float64
1,3,costa_rica,fuel_electricity,1002,36.287
2,6,costa_rica,fuel_oil,1002,72.5739


In [88]:
i = 37
print("table_name: $(tabs_populated[i])")
df_cur = get(dict_tables, tabs_populated[i], nothing);
df_cur

table_name: CapacityFactor

,id,r,t,l,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,pp_geothermal,fallwd0,1000,0.72
2,2,costa_rica,pp_geothermal,fallwd0,1001,0.72
3,3,costa_rica,pp_geothermal,fallwd0,1002,0.72
4,4,costa_rica,pp_geothermal,fallwd0,1003,0.72
5,5,costa_rica,pp_geothermal,fallwd0,1004,0.72
6,6,costa_rica,pp_geothermal,fallwd0,1005,0.72
7,7,costa_rica,pp_geothermal,fallwd0,1006,0.72
8,8,costa_rica,pp_geothermal,fallwd0,1007,0.72
9,9,costa_rica,pp_geothermal,fallwd0,1008,0.72


In [8]:
#DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
#sort(collect(keys(dict_tables)))
#get(dict_tables, "CapacityFactor", nothing)

In [22]:
DBInterface.execute(db, "select * from CapitalCostStorage where (id in (1, 2, 3)) and (y in ('1005'))") |> DataFrame

,id,r,s,y,val
,Int64?,String?,String?,String?,Float64?


In [91]:
fp_test = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))


"/Users/jsyme/.julia/packages/NemoMod/vg6Tx/test/storage_test.sqlite"

In [115]:

model = Model(Gurobi.Optimizer)
#set_optimizer_attribute(model, "Solver", "cplex")
@time NemoMod.calculatescenario(
    fp_sqlite_nemomod_db_tmp; 
    jumpmodel = model,
    numprocs = 1
)

2022-31-Oct 06:35:20.388 Started modeling scenario.
2022-31-Oct 06:35:20.389 Validated run-time arguments.
2022-31-Oct 06:35:20.392 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2022-31-Oct 06:35:20.399 Dropped pre-existing result tables from database.
2022-31-Oct 06:35:20.399 Verified that transmission modeling is not enabled.
2022-31-Oct 06:35:20.482 Created parameter views and indices.
2022-31-Oct 06:35:20.487 Created temporary tables.
2022-31-Oct 06:35:22.241 Executed core database queries.
2022-31-Oct 06:35:22.245 Defined dimensions.
2022-31-Oct 06:35:22.264 Defined demand variables.
2022-31-Oct 06:35:22.277 Defined storage variables.
2022-31-Oct 06:35:22.279 Defined capacity variables.
2022-31-Oct 06:35:25.271 Defined activity variables.
2022-31-Oct 06:35:25.275 Defined costing variables.
2022-31-Oct 06:35:25.276 Defined reserve margin variables.
2022-31-Oct 06:35:25.302 Defined emissi

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [114]:
typeof(model)

Model

In [106]:
MOI.compute_conflict!(model.moi_backend.optimizer.model)
MOI.get(model.moi_backend, Gurobi.ConstraintConflictStatus(), con1.index)



IIS runtime: 0.02 seconds (0.00 work units)


LoadError: Gurobi Error 10015: Cannot compute IIS on a feasible model